In [ ]:
import csv
import time 
from lxml import html
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

url = 'https://tw.hotels.com/'
key = '台北市'
checkin = '2019-04-27'
checkout = '2019-04-29'

driver = None

In [ ]:
def start_driver():
    global driver
    print('啟動 WebDriver...')
    driver = webdriver.Chrome('./chromedriver')
    driver.implicitly_wait(10)

In [ ]:
def close_driver():
    global driver
    driver.quit()
    print('關閉 WebDriver ...')

In [ ]:
def get_page(url):
    global driver
    print('取得網頁...')
    driver.get(url)
    time.sleep(2)

In [ ]:
def search_hotel(searchKey, checkin, checkout):
    global driver
    searchEle = driver.find_element_by_xpath('//input[contains(@id,"destination")]')
    checkInEle = driver.find_element_by_xpath('//input[contains(@class,"check-in")]')
    checkOutEle = driver.find_element_by_xpath('//input[contains(@class,"check-out")]')

    if searchEle and checkInEle and checkOutEle:    
        actions = ActionChains(driver)     # 關閉彈出框
        actions.send_keys(Keys.TAB)
        actions.send_keys(Keys.TAB)
        actions.send_keys(Keys.TAB)
        actions.send_keys(Keys.TAB)
        actions.send_keys(Keys.ENTER)
        actions.perform()
               
        searchEle.send_keys(searchKey)  # 輸入搜尋條件
        searchEle.send_keys(Keys.TAB)        
        checkInEle.clear()
        checkInEle.send_keys(checkin)
        checkOutEle.clear()
        checkOutEle.send_keys(checkout)
        checkOutEle.send_keys(Keys.ENTER)  # 送出搜尋    
        
        time.sleep(10)
        menu = driver.find_element_by_xpath('//*[@id="enhanced-sort"]/li[5]/a')
        if menu:
            actions = ActionChains(driver)
            actions.move_to_element(menu)
            actions.perform()
            price = driver.find_element_by_xpath('//*[@id="sort-submenu-price"]/li[2]/a')
            if price:
                price.click()
                time.sleep(10)
                return True
    return False

In [ ]:
def grab_hotels():
    global driver
    hotels = driver.find_elements_by_xpath("//section[@class='hotel-wrap']")   
    found_hotels = [["旅館名稱",'價格','星級','地址']]
    i = 0
    for hotel in hotels:
        hotelName = driver.find_elements_by_xpath('//h3/a')
        #print(hotelName[i].get_attribute('innerHTML'))
        Name = hotelName[i].text
        price = driver.find_elements_by_xpath('//div[@class="price"]/a/ins')
        try:
            if price:
                price = price[i].text.replace(',','').strip()
            else:
                price = driver.find_elements_by_xpath('//div[@class="price"]/a')
                price = price[i].text.replace(",","").strip()
        except IndexError:
            price = '已訂滿'
        rating = driver.find_elements_by_xpath('//span[@class="star-rating-text"]')
        try:
            if rating:
                rating = rating[i].text
            else:
                rating = driver.find_elements_by_xpath('//span[@class="star-rating-text star-rating-text-strong"]')
                rating = rating[i].text
        except IndexError:
            rating = 'None'
        address = driver.find_elements_by_xpath('//span[@dir="ltr"]')
        if address:
            address = address[i].text
            
        item = [Name,price,rating,address]
        found_hotels.append(item)
        i += 1
    return found_hotels

In [ ]:
def parse_hotels(url,searchKey, checkin, checkout):
    start_driver()
    get_page(url)
    if search_hotel(searchKey,checkin,checkout):
        hotels = grab_hotels()
        close_driver()
        return hotels
    else:
        print("Error...")
        return []

In [ ]:
def image_scral():
    global driver
    figure = driver.find_elements_by_class_name('image')
    for i in range(len(figure)):
        print(figure[i].a)

In [ ]:
def save_to_csv(items,file):
    with open(file,'w+',newline="",encoding='utf-8-sig') as f:
        writer = csv.writer(f)
        for item in items:
            writer.writerow(item)

In [ ]:
if __name__ == '__main__':
    hotels = parse_hotels(url,key,checkin,checkout)
    n = 0
    for hotel in hotels:
        if n == 0:
            n +=1
            continue       
        print("{}. {}\n 價格:{} 星級:{}\n  地址:{}".format(n,hotel[0],hotel[1],hotel[2],hotel[3]))
        n +=1
    #image_scral()    
    save_to_csv(hotels,'hotels.csv')